# Initial Setup

The project repository is mounted from Google Drive and added to the Python path to allow clean imports from the src module. The dataset is copied to the local Colab filesystem to improve I/O performance during training. All global settings (random seed, device selection, paths, batch sizes) are defined once and reused across the notebook to ensure consistency and reproducibility.

Weights & Biases is initialized for experiment tracking, and all training stages use the same precomputed dataset statistics and DataLoaders for fair comparison across models.

In [ ]:
import sys
from pathlib import Path

from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/Colab Notebooks/Applied Computer Vision/Applied-Computer-Vision-Projects/Multimodal_Learning_02/"

PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

In [ ]:
# Install dependencies
%%capture
%pip install --no-cache-dir -r requirements.txt

In [ ]:
# %pip install wandb fiftyone==1.10.0 sympy==1.12 torch==2.9.0 torchvision==0.20.0 numpy open-clip-torch

In [ ]:
import os
from google.colab import userdata

import torch
import torch.nn as nn
import torchvision.transforms.v2 as transforms
from torch.optim import Adam

import wandb
import matplotlib.pyplot as plt

In [ ]:
!rm -rf /content/data
!cp -r "/content/drive/MyDrive/Colab Notebooks/Applied Computer Vision/Applied-Computer-Vision-Projects/Multimodal_Learning_02/data_transformed" /content/data
# !cp -r "/content/drive/MyDrive/Colab Notebooks/Applied Computer Vision/Applied-Computer-Vision-Projects/Multimodal_Learning_02/data/assessment" /content/data

In [ ]:
from src.config import (SEED, NUM_WORKERS, BATCH_SIZE, IMG_SIZE, N, CLASSES,
                        NUM_CLASSES, LABEL_MAP, RAW_DATA, CHECKPOINTS, DEVICE, VALID_BATCHES)
from src.utility import set_seeds, init_wandb
from src.datasets import compute_dataset_mean_std, get_dataloaders
from src.training import compute_class_weights, get_early_inputs, get_inputs, train_model
from src.visualization import build_pairwise_downsampling_tables, plot_losses
from src.models import EmbedderMaxPool, EmbedderStrided, EarlyFusionModel, ConcatIntermediateNet, AddIntermediateNet, MatmulIntermediateNet, HadamardIntermediateNet, LateNet

In [ ]:
# Fusion specific constants
EPOCHS = 15
LR = 0.0001

In [ ]:
# Usage: Call this function at the beginning and before each training phase
set_seeds(SEED)

In [ ]:
# Load W&B API key from Colab Secrets and make it available as env variable
wandb_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_key
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: michele-marschner (michele-marschner-university-of-potsdam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Loading and preparation of Data

This section computes normalization statistics, defines the RGB+LiDAR transforms, and prepares the training, validation, and test DataLoaders. Both downsampling strategies (MaxPool and Stride) receive identical input preprocessing to guarantee a fair ablation comparison.

In [ ]:
# Calculates mean and standard deviation of the rgb train data
# for different dataset (or change in train data) recalculate mean and standard deviation
# mean, std = compute_dataset_mean_std(root_dir=RAW_DATA, img_size=IMG_SIZE)

Scanning dataset in /content/drive/MyDrive/Colab Notebooks/Applied Computer Vision/Applied-Computer-Vision-Projects/Multimodal_Learning_02/data...
cubes: 2501 RGB files found. Matching XYZA...


spheres: 9999 RGB files found. Matching XYZA...


Preloading LiDAR XYZA tensors into RAM...


Loading XYZA:   0%|          | 38/12500 [00:28<2:35:57,  1.33it/s]

In [ ]:
img_transforms = transforms.Compose([
    transforms.ToImage(),   # Scales data into [0,1]
    transforms.Resize(IMG_SIZE),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(([0.0051, 0.0052, 0.0051, 1.0000]), ([5.8023e-02, 5.8933e-02, 5.8108e-02, 2.4509e-07])) 
    # transforms.Normalize(mean.tolist(), std.tolist())     # uncomment for different dataset (or change in train data)
])

In [ ]:
set_seeds(SEED)

train_data, train_dataloader, valid_data, val_dataloader, test_data, test_dataloader = get_dataloaders(
    str(RAW_DATA),
    VALID_BATCHES,
    test_frac=0.15,
    batch_size=BATCH_SIZE,
    img_transforms=img_transforms,
    num_workers=NUM_WORKERS,
    seed=SEED
)

for i, sample in enumerate(train_data):
    print(i, *(x.shape for x in sample))
    break

# Model Training

In this section we train multiple fusion models using either MaxPool-based embedders or StridedConv-based embedders. For each architecture, we initialize the optimizer, create a W&B run, train the model, and save checkpoints. The goal is to isolate the impact of replacing MaxPool with stride-based downsampling while keeping all other components constant.

In [ ]:
FEATURE_DIM = 128

set_seeds(SEED)

class_weights = compute_class_weights(train_data, NUM_CLASSES).to(DEVICE)
loss_func = nn.CrossEntropyLoss(weight=class_weights.to(DEVICE))

metrics = {}   # store losses for each model

# Defines fusion models to train and compare
models_to_train = {
    "early_fusion_pool": EarlyFusionModel(in_ch=8, output_dim=2, embedder_cls=EmbedderMaxPool).to(DEVICE),
    "early_fusion_stride": EarlyFusionModel(in_ch=8, output_dim=2, embedder_cls=EmbedderStrided).to(DEVICE),
    "intermediate_fusion_concat_pool": ConcatIntermediateNet(4, 4, output_dim=NUM_CLASSES, feature_dim=FEATURE_DIM, embedder_cls=EmbedderMaxPool).to(DEVICE),
    "intermediate_fusion_concat_stride": ConcatIntermediateNet(4, 4, output_dim=NUM_CLASSES, feature_dim=FEATURE_DIM, embedder_cls=EmbedderStrided).to(DEVICE),
    "late_fusion_pool": LateNet(4, 4, output_dim=NUM_CLASSES, embedder_cls=EmbedderMaxPool).to(DEVICE),
    "late_fusion_stride": LateNet(4, 4, output_dim=NUM_CLASSES, embedder_cls=EmbedderStrided).to(DEVICE)
}

# === Main experiment loop over all fusion strategies ===
for name, model in models_to_train.items():
  model_save_path = CHECKPOINTS

  # Number of trainable parameters (for the comparison table)
  num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

  opt = Adam(model.parameters(), lr=LR)

  # Initialize a new Weights & Biases run for this model.
  init_wandb(
      model=model,
      name=name,
      fusion_name=name,
      num_params=num_params,
      opt_name = opt.__class__.__name__)

  # Choose the proper input function depending on the fusion strategy:
  if name.startswith("early_fusion"):
    input_fn = get_early_inputs
  else:
    input_fn = get_inputs

  results = train_model(
    model=model,
    optimizer=opt,
    input_fn=input_fn,
    epochs=EPOCHS,
    loss_fn=loss_func,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    model_save_path=model_save_path,
    target_idx=-1,   # last element in batch is target
    log_to_wandb=True,
    device=DEVICE
  )

  metrics[name] = results

  # End wandb run before starting the next model
  wandb.finish()

# Evaluation

This section plots validation losses for all trained models and generates downsampling comparison tables. These tables summarize performance differences between MaxPool and StridedConv variants across early, intermediate, and late fusion methods. All results are logged to W&B for easy inspection and reproducibility.

In [ ]:
name_map = {
    "early_fusion": "Early Fusion",
    "intermediate_fusion_concat": "Intermediate (Concat)",
    "late_fusion": "Late Fusion",
}

In [ ]:
loss_dict = {name: m["valid_losses"] for name, m in metrics.items()}
fig = plot_losses(loss_dict, title="Validation Loss per Model")
plt.show()

tables = build_pairwise_downsampling_tables(metrics, name_map)

In [ ]:
# logs comparison tables to wandb
for base, df in tables.items():
    wandb.init(
        project="cilp-extended-assessment",
        name=f"downsampling_comparison_{base}",
        job_type="analysis",
    )
    wandb.log({f"task4_downsampling_{base}": wandb.Table(dataframe=df)})
    wandb.finish()

## When to Use

This final section provides guidance on when each downsampling method—MaxPool or Strided Conv—is preferable based on the experiment results. It summarizes strengths, limitations, and practical scenarios where each approach performs best, helping translate the ablation findings into actionable design choices.

**Early Fusion:**
* Aligned, closely related low-level modalities and comparable features
* Simple setup; avoid if sensors are noisy

**Intermediate Fusion:**
* Modalities with different structure that benefit from separate early processing in order to learn modality-specific features   
* best overall balance of performance and flexibility

**Late Fusion:**
* Strong, independent unimodal predictors, to combine their strengths
* ideal for heterogeneous or missing modalities
* robust fallback when one modality fails